# Model Evaluation

# 1. Data Preprocessing
The dataset has 2,815,677 rows

3.8 million rows with 55 columns. Please use an appropriate sample size before plotting. <br>
I will first need to transform some of the integer-like categorical values. The keys are located in 'US_Birth_columns' text file. <br>
In addition, I will need to engineer new variables to better enhance the R2 score.

### Libraries to import:

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from scipy import stats
%matplotlib inline
import random
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor

from sklearn.feature_selection import RFECV
from sklearn.svm import SVR

from sklearn.linear_model import LassoCV

### Import Data:

In [11]:
columns = ['dob_mm', 'dob_tt', 'dob_wk', 'bfacil', 'mager', 'mbstate_rec', 'restatus', 'mrace6', 'mhisp_r', 'mracehisp', 'dmar', 'meduc', 'fagecomb',
          'frace6', 'fhisp_r', 'fracehisp', 'feduc', 'priorlive', 'priordead', 'priorterm', 'illb_r', 'ilop_r', 'ilp_r', 'precare', 'previs', 'cig_0',
          'cig_1', 'cig_2', 'cig_3', 'm_ht_in', 'f_m_ht', 'bmi', 'pwgt_r', 'dwgt_r', 'wtgain', 'rf_pdiab', 'rf_gdiab', 'rf_phype', 'rf_ghype', 'rf_ehype',
          'rf_ppterm', 'rf_inftr', 'rf_fedrg', 'rf_artec', 'rf_cesar', 'rf_cesarn', 'no_risks', 'ip_gon', 'ip_syph', 'ip_chlam', 'ip_hepatb', 'ip_hepatc', 
          'no_infec', 'ob_ecvs', 'ob_ecvf', 'ld_indl', 'ld_augm', 'ld_ster', 'ld_antb', 'ld_chor', 'ld_anes', 'no_lbrdlv', 'me_pres', 'me_rout', 'me_trial',
          'rdmeth_rec', 'mm_mtr', 'mm_plac', 'mm_rupt', 'mm_uhyst', 'mm_aicu', 'no_mmorb', 'attend', 'mtran', 'pay', 'dplural', 'sex', 'combgest',
          'dbwt']

In [12]:
n = 3_801_534
s = 750_000
skip = sorted(random.sample(range(1, n + 1), n-s))

In [13]:
df = pd.read_csv('natl2018.csv.zip', low_memory=False, usecols=columns, skiprows=skip)
# df = df.sample(n=250_000, random_state=42)
df.shape

(750000, 79)

In [14]:
df.head()

,dob_mm,dob_tt,dob_wk,bfacil,mager,mbstate_rec,restatus,mrace6,mhisp_r,mracehisp,dmar,meduc,fagecomb,frace6,fhisp_r,fracehisp,feduc,priorlive,priordead,priorterm,illb_r,ilop_r,ilp_r,precare,previs,cig_0,cig_1,cig_2,cig_3,m_ht_in,f_m_ht,bmi,pwgt_r,dwgt_r,wtgain,rf_pdiab,rf_gdiab,rf_phype,rf_ghype,rf_ehype,rf_ppterm,rf_inftr,rf_fedrg,rf_artec,rf_cesar,rf_cesarn,no_risks,ip_gon,ip_syph,ip_chlam,ip_hepatb,ip_hepatc,no_infec,ob_ecvs,ob_ecvf,ld_indl,ld_augm,ld_ster,ld_antb,ld_chor,ld_anes,no_lbrdlv,me_pres,me_rout,me_trial,rdmeth_rec,mm_mtr,mm_plac,mm_rupt,mm_uhyst,mm_aicu,no_mmorb,attend,mtran,pay,dplural,sex,combgest,dbwt
0,1,2145,1,1,35,1,1,10,0,1,1.0,4,35,1,0,1,8,2,0,0,138,888,138,3,10,10,0,0,0,63,1,25.7,145,193,48,N,N,N,N,N,N,N,X,X,N,0,1,N,N,N,N,N,1,N,N,N,Y,N,N,N,Y,0,1,1,X,1,N,N,N,N,N,1,1,N,2,1,M,40,3910
1,1,802,4,1,39,1,1,10,0,1,1.0,4,34,3,0,3,4,0,0,0,888,888,888,2,12,0,0,0,0,62,1,23.8,130,175,45,N,N,N,N,N,N,N,X,X,N,0,1,N,N,N,N,N,1,N,N,N,N,N,Y,N,Y,0,1,4,Y,3,N,N,N,N,N,1,1,N,4,1,M,38,3013
2,1,750,7,1,27,1,1,10,0,1,1.0,6,27,1,0,1,3,0,0,0,888,888,888,2,8,0,0,0,0,64,1,21.3,124,138,14,N,N,N,N,N,N,N,X,X,N,0,1,N,N,N,N,N,1,N,N,N,N,N,N,N,Y,0,1,1,X,1,N,N,N,N,N,1,1,N,3,1,M,43,3495
3,1,1501,3,1,32,1,1,10,0,1,1.0,6,30,1,0,1,4,1,0,0,33,888,33,4,13,0,0,0,0,68,1,29.2,192,220,28,N,N,N,N,N,N,N,X,X,N,0,1,N,N,N,N,N,1,N,N,N,N,N,N,N,N,1,1,1,X,1,N,N,N,N,N,1,1,N,5,1,M,41,4153
4,1,40,5,1,34,1,2,10,9,8,1.0,6,33,1,9,8,7,0,0,0,888,888,888,7,8,0,0,0,0,65,1,25.6,154,184,30,N,N,N,N,N,N,Y,N,Y,N,0,0,N,N,N,N,N,1,N,N,N,N,N,N,Y,Y,0,1,4,Y,3,N,N,N,N,N,1,1,N,2,1,F,41,3840


In [19]:
def data_cleaning_births(_df):
    """
    Preprocessing columns by translating using column dictionary.
    The order of columns follows the same order as column dictionary.
    """
    
    # Birth Month, leave numbers as is; 12 = December, 01 = January
    _df['dob_mm'] = _df['dob_mm'] 
    
    
    # Time of Birth, dropping unknown
    _df.drop(df[df['dob_tt'].eq(9999)].index, inplace=True)    
    
    
    # Birth Day of Week, leave as is; 7 = Saturday, 1 = Sunday
    _df['dob_wk'] = _df['dob_wk']
    
    # Birth Place; Home(intended) = Home(not intended)
    conditions = [
        _df['bfacil'].eq(1),
        _df['bfacil'].eq(2),
        _df['bfacil'].eq(3),
        _df['bfacil'].eq(4), 
        _df['bfacil'].eq(5),
        _df['bfacil'].eq(6),
        _df['bfacil'].eq(7), 
    ]
    choices = [
        'Hospital',
        'BirthCenter',
        'Home',
        'Home',
        'Home',
        'Clinic',
        'Other'
    ]
    _df['bfacil'] = np.select(conditions, choices, 'Unknown')
    
    # Mother's Single Years of Age; note 0 = 10-12, and 50 = 50 and over
    _df['mager'] = _df['mager']
    
    # Mother's Nativity
    conditions = [
        _df['mbstate_rec'].eq(1),
        _df['mbstate_rec'].eq(2)
    ]
    choices = [
        'BornInUSA',
        'BornOutUSA'
    ]
    _df['mbstate_rec'] = np.select(conditions, choices, 'Unknown')
    
    
    # Residence Status
    conditions = [
        _df['restatus'].eq(1),
        _df['restatus'].eq(2),
        _df['restatus'].eq(3)
    ]
    choices = [
        'Resident',
        'IntrastateNonresident',
        'InterstateNonresident'
    ]
    _df['restatus'] = np.select(conditions, choices, 'Foreign')
    
    # Mother's Race Recode 6
    conditions = [
        _df['mrace6'].eq(10),
        _df['mrace6'].eq(20),
        _df['mrace6'].eq(30),
        _df['mrace6'].eq(41),
        _df['mrace6'].eq(51)
    ]
    choices = [
        'White',
        'Black',
        'AIAN',
        'Asian',
        'NHOPI',
    ]
    
    _df['mrace6'] = np.select(conditions, choices, 'Multi')
    
    # Mother's Hispanic Origin
    conditions = [
        _df['mhisp_r'].eq(0),
        _df['mhisp_r'].eq(1),
        _df['mhisp_r'].eq(2),
        _df['mhisp_r'].eq(3),
        _df['mhisp_r'].eq(4),
        _df['mhisp_r'].eq(5),
        _df['mhisp_r'].eq(6)
    ]
    choices = [
        'Non-Hispanic',
        'Mexican',
        'Puerto Rican',
        'Cuban',
        'CentralSouthAmerican',
        'OtherHispanic',
        'Dominican'
    ]
    _df['mhisp_r'] = np.select(conditions, choices, 'Unknown')
    
    # Mother's Race/Hispanic Origin
    conditions = [
        _df['mracehisp'].eq(1),
        _df['mracehisp'].eq(2),
        _df['mracehisp'].eq(3),
        _df['mracehisp'].eq(4),
        _df['mracehisp'].eq(5),
        _df['mracehisp'].eq(6),
        _df['mracehisp'].eq(7),
    ]
    choices = [
        'NonHispanicWhite',
        'NonHispanicBlack',
        'NonHispanicAIAN',
        'NonHispanicAsian',
        'NonHispanicNHOPI',
        'NonHispanicMulti',
        'Hispanic'
    ]
    _df['mracehisp'] = np.select(conditions, choices, 'Unknown')
    
    
    # Marital Status
    _df['dmar'] = np.where(_df['dmar'].eq(1), 'Married', 'Unmarried')
    
    # Mother's Education
    conditions = [
        _df['meduc'].eq(1),
        _df['meduc'].eq(2),
        _df['meduc'].eq(3),
        _df['meduc'].eq(4),
        _df['meduc'].eq(5),
        _df['meduc'].eq(6),
        _df['meduc'].eq(7),
        _df['meduc'].eq(8)
    ]
    choices = [
        'JuniorHighIncomplete',
        'HighSchoolIncomplete',
        'HighSchoolComplete',
        'CollegeIncomplete',
        'AssociateDegree',
        'BachelorDegree',
        'MasterDegree',
        'Doctorate'
    ]
    _df['meduc'] = np.select(conditions, choices, 'Unknown')
    
    # Father's combined Age: imputing missing fathers age with average
    _df['fagecomb'] = np.where(_df['fagecomb'].eq(99), 30, _df['fagecomb'])
    
    # Father's Race recode
    conditions = [
        _df['frace6'].eq(1),
        _df['frace6'].eq(2),
        _df['frace6'].eq(3),
        _df['frace6'].eq(4),
        _df['frace6'].eq(5),
        _df['frace6'].eq(6),
    ]
    choices = [
        'White',
        'Black',
        'AIAN',
        'Asian',
        'NHOPI',
        'Multi'
    ]
    _df['frace6'] = np.select(conditions, choices, 'Unknown')
    
    # Father's Hispanic Origin
    conditions = [
        _df['fhisp_r'].eq(0),
        _df['fhisp_r'].eq(1),
        _df['fhisp_r'].eq(2),
        _df['fhisp_r'].eq(3),
        _df['fhisp_r'].eq(4),
        _df['fhisp_r'].eq(5),
        _df['fhisp_r'].eq(6)
    ]
    choices = [
        'NonHispanic',
        'Mexican',
        'PuertoRican',
        'Cuban',
        'CentralSouthAmerican',
        'Dominican',
        'OtherHispanic'
    ]
    _df['fhisp_r'] = np.select(conditions, choices, 'Unknown')
    
    # Father's Race/Hispanic Origin
    conditions = [
        _df['fracehisp'].eq(1),
        _df['fracehisp'].eq(2),
        _df['fracehisp'].eq(3),
        _df['fracehisp'].eq(4),
        _df['fracehisp'].eq(5),
        _df['fracehisp'].eq(6),
        _df['fracehisp'].eq(7),
        _df['fracehisp'].eq(8)
    ]
    choices = [
        'NonHispanicWhite',
        'NonHispanicBlack',
        'NonHispanicAIAN',
        'NonHispanicAsian',
        'NonHispanicNHOPI',
        'NonHispanicMulti',
        'Hispanic',
        'Unknown'
    ]
    _df['fracehisp'] = np.select(conditions, choices, 'Unknown')
    
    # Father's Education
    conditions = [
        _df['feduc'].eq(1),
        _df['feduc'].eq(2),
        _df['feduc'].eq(3),
        _df['feduc'].eq(4),
        _df['feduc'].eq(5),
        _df['feduc'].eq(6),
        _df['feduc'].eq(7),
        _df['feduc'].eq(8)
    ]
    choices = [
        'JuniorHighIncomplete',
        'HighSchoolIncomplete',
        'HighSchoolComplete',
        'CollegeIncomplete',
        'AssociateDegree',
        'BachelorDegree',
        'MasterDegree',
        'Doctorate'
    ]
    _df['feduc'] = np.select(conditions, choices, 'Unknown')
    
    # Prior Births Now Living
    conditions = [
        _df['priorlive'].eq(0),
        _df['priorlive'].eq(1),
        _df['priorlive'].eq(99)
    ]
    choices = [
        'None',
        'One',
        'Unknown'
    ]
    _df['priorlive'] = np.select(conditions, choices, 'Multi')
    
    # Prior Births Now Dead
    conditions = [
        _df['priordead'].eq(0),
        _df['priordead'].eq(1),
        _df['priordead'].eq(99)
    ]
    choices = [
        'None',
        'One',
        'Unknown'
    ]
    _df['priordead'] = np.select(conditions, choices, 'Multi')
    
    # Prior Other Terminations
    conditions = [
        _df['priorterm'].eq(0),
        _df['priorterm'].eq(1),
        _df['priorterm'].eq(99)
    ]
    choices = [
        'None',
        'One',
        'Unknown'
    ]
    _df['priorterm'] = np.select(conditions, choices, 'Multi')
    
    
    # Interval Since Last Live Birth Recode
    _df.drop(df[df['illb_r'].eq(999)].index, inplace=True)    
    _df['illb_r'] = np.where(_df['illb_r'].eq(888), 0, _df['illb_r'])
    
    # Interval since last other Pregnancy
    _df.drop(df[df['ilop_r'].eq(999)].index, inplace=True)    
    _df['ilop_r'] = np.where(_df['ilop_r'].eq(888), 0, _df['ilop_r'])
    
    # Interval since Last pregnancy
    _df.drop(df[df['ilp_r'].eq(999)].index, inplace=True)    
    _df['ilp_r'] = np.where(_df['ilp_r'].eq(888), 0, _df['ilp_r'])
    
    # Month Prenatal Care Began
    _df['precare'] = np.where(_df['precare'].eq(99), 0, _df['precare'])
    
    # Number of Prenatal Visits
    _df['previs'] = np.where(_df['previs'].eq(99), 0, _df['previs'])
    
    # Cigarettes before Pregnancy
    conditions = [
        _df['cig_0'].eq(0),
        _df['cig_0'].eq(99)
    ]
    choices = [
        'None',
        'Unknown'
    ]
    _df['cig_0'] = np.select(conditions, choices, 'Yes')
    
    # Cigarettes 1st Trimester
    conditions = [
        _df['cig_1'].eq(0),
        _df['cig_1'].eq(99)
    ]
    choices = [
        'None',
        'Unknown'
    ]
    _df['cig_1'] = np.select(conditions, choices, 'Yes')
    
    # Cigarettes 2nd Trimester
    conditions = [
        _df['cig_2'].eq(0),
        _df['cig_2'].eq(99)
    ]
    choices = [
        'None',
        'Unknown'
    ]
    _df['cig_2'] = np.select(conditions, choices, 'Yes')
    
    # Cigarettes 3rd Trimester
    conditions = [
        _df['cig_3'].eq(0),
        _df['cig_3'].eq(99)
    ]
    choices = [
        'None',
        'Unknown'
    ]
    _df['cig_3'] = np.select(conditions, choices, 'Yes')
    
    
    # Mother's Height in Total Inches: imputing average height if missing
    _df['m_ht_in'] = np.where(_df['m_ht_in'].eq(99), 64, _df['m_ht_in'])
    
    # Height Reported
    _df['f_m_ht'] = _df['f_m_ht']
    
    
    # Body Mass Index: if missing, impute mean
    _df['bmi'] = np.where(_df['bmi'].eq(99), 27.9, _df['bmi'])
    
    # Pre-pregnancy weight
    _df['pwgt_r'] = np.where(_df['pwgt_r'].eq(999), 164, _df['pwgt_r'])
    
    
    # Delivery Weight Recode
    _df['dwgt_r'] = np.where(_df['dwgt_r'].eq(999), 193, _df['dwgt_r'])
    
    
    # Weight Gain
    _df['wtgain'] = np.where(_df['wtgain'].eq(99), _df['dwgt_r'] - _df['pwgt_r'], _df['wtgain'])
    
    # Pre-pregnancy Diabetes
    # .... skipping columns leaving as is. Y = Yes, N= No, U = Unknown, X = Not Applicable
    # Previous Cesarean
    
    # Number of Previous Cesareans
    conditions = [
        _df['rf_cesarn'].eq(0),
        _df['rf_cesarn'].eq(1),
        _df['rf_cesarn'].eq(99),
    ]
    choices = [
        'None',
        'One',
        'Unknown'
    ]
    _df['rf_cesarn'] = np.select(conditions, choices, 'Multi')
    
    
    # No Risk Factors Reported
    conditions = [
        _df['no_risks'].eq(1),
        _df['no_risks'].eq(0)
    ]
    choices = [
        'Y',
        'N'
    ]
    _df['no_risks'] = np.select(conditions, choices, 'U')
    
    # IP_GON Gonorrhea
    # ... skipping columns, leaving as is. Y= Yes, N= No, U= Unknown
    # IP_HEPC Hepatitis c
    
    # No Infections Reported
    conditions = [
        _df['no_infec'].eq(1),
        _df['no_infec'].eq(0)
    ]
    choices = [
        'Y',
        'N'
    ]
    _df['no_infec'] = np.select(conditions, choices, 'U')
    
    # OB_ECVS Successful External Cephalic Version
    # ... skipping columns
    # LD_ANES Anesthesia
    
    # No Characteristics of Labor Reported
    conditions = [
        _df['no_lbrdlv'].eq(1),
        _df['no_lbrdlv'].eq(0)
    ]
    choices = [
        'Y',
        'N'
    ]
    _df['no_lbrdlv'] = np.select(conditions, choices, 'U')
    
    
    # Fetal Presentation at Delivery
    conditions = [
        _df['me_pres'].eq(1),
        _df['me_pres'].eq(2),
        _df['me_pres'].eq(3)
    ]
    choices = [
        'Cephalic',
        'Breech',
        'Other'
    ]
    _df['me_pres'] = np.select(conditions, choices, 'Unknown')
    
    
    # Final Route and Method of Delivery
    conditions = [
        _df['me_rout'].eq(1),
        _df['me_rout'].eq(2),
        _df['me_rout'].eq(3),
        _df['me_rout'].eq(4)
    ]
    choices = [
        'Spontaneous',
        'Forceps',
        'Vacuum',
        'Cesarean'
    ]
    _df['me_rout'] = np.select(conditions, choices, 'Unknown')
    
    # Trial of Labor Attempted
    # skip
    
    # Delivery Method Recode
    conditions = [
        _df['rdmeth_rec'].eq(1),
        _df['rdmeth_rec'].eq(2),
        _df['rdmeth_rec'].eq(3),
        _df['rdmeth_rec'].eq(4),
        _df['rdmeth_rec'].eq(5),
        _df['rdmeth_rec'].eq(6)
    ]
    choices = [
        'Vaginal',
        'VaginalAfterCSection',
        'PrimaryCSection',
        'RepeatCSection',
        'Vaginal',
        'PrimaryCSection'
    ]
    _df['rdmeth_rec'] = np.select(conditions, choices, 'Unknown')
    
    # Maternal Transfusion
    # skip
    # Admit to Intensive Care
    
    # No Maternal Morbidty Reported
    conditions = [
        _df['no_mmorb'].eq(1),
        _df['no_mmorb'].eq(0),
    ]
    choices = [
        'Yes',
        'No'
    ]
    _df['no_mmorb'] = np.select(conditions, choices, 'Unknown')
    
    # Attendant at Birth
    conditions = [
        _df['attend'].eq(1),
        _df['attend'].eq(2),
        _df['attend'].eq(3),
        _df['attend'].eq(4),
        _df['attend'].eq(5)
    ]
    choices = [
        'DoctorMedicine',
        'DoctorOsteopathy',
        'NurseMidwife',
        'OtherMidwife',
        'Other'
    ]
    _df['attend'] = np.select(conditions, choices, 'Unknown')
    
    
    # Mother Transferred
    # skip
    
    # Payment Source for Delivery
    conditions = [
        _df['pay'].eq(1),
        _df['pay'].eq(2),
        _df['pay'].eq(3),
        _df['pay'].eq(4),
        _df['pay'].eq(5),
        _df['pay'].eq(6),
        _df['pay'].eq(8),
    ]
    choices = [
        'Medicaid',
        'PrivateInsurance',
        'SelfPay',
        'IndianHealthService',
        'Tricare',
        'OtherGovernment',
        'Other'
    ]
    _df['pay'] = np.select(conditions, choices, 'Unknown')
    
    
    # Plurality Recode
    # skip
    
    # Sex of Infant
    #skip
    
    # Combined Gestation
    _df['combgest'] = np.where(_df['combgest'].eq(99), 38, _df['combgest'])
    
    # Obstetric Estimate
#     _df['oegest_comb'] = np.where(_df['oegest_comb'].eq(99), 38, _df['oegest_comb'])
    
    # Birth Weight
    _df.drop(_df[_df['dbwt'].eq(9999)].index, inplace=True)
    
    
    return _df

In [20]:
df = data_cleaning_births(df)

In [21]:
df

,dob_mm,dob_tt,dob_wk,bfacil,mager,mbstate_rec,restatus,mrace6,mhisp_r,mracehisp,dmar,meduc,fagecomb,frace6,fhisp_r,fracehisp,feduc,priorlive,priordead,priorterm,illb_r,ilop_r,ilp_r,precare,previs,cig_0,cig_1,cig_2,cig_3,m_ht_in,f_m_ht,bmi,pwgt_r,dwgt_r,wtgain,rf_pdiab,rf_gdiab,rf_phype,rf_ghype,rf_ehype,rf_ppterm,rf_inftr,rf_fedrg,rf_artec,rf_cesar,rf_cesarn,no_risks,ip_gon,ip_syph,ip_chlam,ip_hepatb,ip_hepatc,no_infec,ob_ecvs,ob_ecvf,ld_indl,ld_augm,ld_ster,ld_antb,ld_chor,ld_anes,no_lbrdlv,me_pres,me_rout,me_trial,rdmeth_rec,mm_mtr,mm_plac,mm_rupt,mm_uhyst,mm_aicu,no_mmorb,attend,mtran,pay,dplural,sex,combgest,dbwt
0,1,2145,1,Unknown,35,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,35,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,138,0,138,3,10,Yes,Yes,Yes,Yes,63,1,25.7,145,193,48,N,N,N,N,N,N,N,X,X,N,Multi,U,N,N,N,N,N,U,N,N,N,Y,N,N,N,Y,U,Unknown,Unknown,X,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,M,40,3910
1,1,802,4,Unknown,39,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,34,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,0,0,0,2,12,Yes,Yes,Yes,Yes,62,1,23.8,130,175,45,N,N,N,N,N,N,N,X,X,N,Multi,U,N,N,N,N,N,U,N,N,N,N,N,Y,N,Y,U,Unknown,Unknown,Y,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,M,38,3013
2,1,750,7,Unknown,27,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,27,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,0,0,0,2,8,Yes,Yes,Yes,Yes,64,1,21.3,124,138,14,N,N,N,N,N,N,N,X,X,N,Multi,U,N,N,N,N,N,U,N,N,N,N,N,N,N,Y,U,Unknown,Unknown,X,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,M,43,3495
3,1,1501,3,Unknown,32,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,30,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,33,0,33,4,13,Yes,Yes,Yes,Yes,68,1,29.2,192,220,28,N,N,N,N,N,N,N,X,X,N,Multi,U,N,N,N,N,N,U,N,N,N,N,N,N,N,N,U,Unknown,Unknown,X,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,M,41,4153
4,1,40,5,Unknown,34,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,33,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,0,0,0,7,8,Yes,Yes,Yes,Yes,65,1,25.6,154,184,30,N,N,N,N,N,N,Y,N,Y,N,Multi,U,N,N,N,N,N,U,N,N,N,N,N,N,Y,Y,U,Unknown,Unknown,Y,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,F,41,3840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749994,12,1103,2,Unknown,24,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,37,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,32,61,32,2,13,Yes,Yes,Yes,Yes,66,1,33.9,210,225,15,N,Y,N,N,N,N,N,X,X,Y,Multi,U,N,N,N,N,N,U,N,N,N,N,N,N,N,Y,U,Unknown,Unknown,Y,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,F,40,3340
749995,12,316,1,Unknown,34,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,43,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,102,0,102,1,10,Yes,Yes,Yes,Yes,62,1,21.4,117,162,45,N,N,N,N,N,N,N,X,X,Y,Multi,U,N,N,N,N,N,U,N,N,Y,N,N,Y,N,Y,U,Unknown,Unknown,N,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,F,39,4240
749996,12,716,1,Unknown,24,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,25,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,25,0,25,4,14,Yes,Yes,Yes,Yes,63,1,23.4,132,152,20,N,N,N,N,N,N,N,X,X,N,Multi,U,N,N,N,N,N,U,N,N,N,Y,N,N,N,Y,U,Unknown,Unknown,X,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,F,38,2800
749998,12,820,2,Unknown,27,Unknown,Foreign,Multi,Unknown,Unknown,Unmarried,Unknown,39,Unknown,Unknown,Unknown,Unknown,Multi,Multi,Multi,0,0,0,2,14,Yes,Yes,Yes,Yes,64,1,28.0,163,179,16,N,N,N,N,N,N,N,X,X,N,Multi,U,N,N,N,N,N,U,N,N,N,N,N,N,N,Y,U,Unknown,Unknown,X,Unknown,N,N,N,N,N,Unknown,Unknown,N,Unknown,1,F,40,3555


# Model Evaluation
This project will in the end be about coefficient explanation. However, having a section for model evaluation will keep me sane. I will split the dataset into a training set and test set and compare r2 scores on the test set.

In [64]:
X = df.drop(columns=['dbwt'])
y = df['dbwt']
X.drop(columns=['dob_tt'], inplace=True)


continuous_columns = [
#     'dob_tt', 
    'mager', 'fagecomb', 'illb_r', 'ilop_r', 'ilp_r', 'precare', 'previs', 'm_ht_in', 'bmi', 'pwgt_r','dwgt_r', 'wtgain', 'dplural', 'combgest']
categorical_columns = [x for x in X.columns if x not in continuous_columns]

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Dummy Regressor

In [66]:
dummy_regr = DummyRegressor(strategy='mean')
dummy_regr.fit(X_train, y_train)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [67]:
dummy_regr_rmse = np.sqrt(mean_squared_error(y_test, dummy_regr.predict(X_test)))
dummy_regr_r2 = r2_score(y_true = y_test, y_pred = dummy_regr.predict(X_test))
print('R2 of Dummy Regressor(mean): ', dummy_regr_r2)
print('RMSE of Dummy Regressor(mean): ', dummy_regr_rmse)

R2 of Dummy Regressor(mean):  -6.579376927029479e-05
RMSE of Dummy Regressor(mean):  582.5006739063442


### Lasso Linear

In [68]:
scaler = StandardScaler()
_X_train_cont = scaler.fit_transform(X_train[continuous_columns])
_X_train_cont_df = pd.DataFrame(data = _X_train_cont,
                                index= X_train.index,
                         columns = continuous_columns)

_X_test_cont = scaler.transform(X_test[continuous_columns])
_X_test_cont_df = pd.DataFrame(data = _X_test_cont,
                                index= X_test.index,
                         columns = continuous_columns)

In [69]:
ohe = OneHotEncoder(handle_unknown='ignore')
_X_train_ohe = ohe.fit_transform(X_train[categorical_columns]).toarray()
_X_train_ohe_df = pd.DataFrame(data = _X_train_ohe,
                         index = X_train.index,
                        columns = ohe.get_feature_names(categorical_columns))

_X_test_ohe = ohe.transform(X_test[categorical_columns]).toarray()
_X_test_ohe_df = pd.DataFrame(data = _X_test_ohe,
                         index = X_test.index,
                        columns = ohe.get_feature_names(categorical_columns))


_X_train_merged = _X_train_cont_df.merge(_X_train_ohe_df, left_index=True, right_index=True)
_X_test_merged = _X_test_cont_df.merge(_X_test_ohe_df, left_index=True, right_index=True)
_X_train_merged.head()

,mager,fagecomb,illb_r,ilop_r,ilp_r,precare,previs,m_ht_in,bmi,pwgt_r,dwgt_r,wtgain,dplural,combgest,dob_mm_1,dob_mm_2,dob_mm_3,dob_mm_4,dob_mm_5,dob_mm_6,dob_mm_7,dob_mm_8,dob_mm_9,dob_mm_10,dob_mm_11,dob_mm_12,dob_wk_1,dob_wk_2,dob_wk_3,dob_wk_4,dob_wk_5,dob_wk_6,dob_wk_7,bfacil_Unknown,mbstate_rec_Unknown,restatus_Foreign,mrace6_Multi,mhisp_r_Unknown,mracehisp_Unknown,dmar_Unmarried,meduc_Unknown,frace6_Unknown,fhisp_r_Unknown,fracehisp_Unknown,feduc_Unknown,priorlive_Multi,priordead_Multi,priorterm_Multi,cig_0_Yes,cig_1_Yes,cig_2_Yes,cig_3_Yes,f_m_ht_0,f_m_ht_1,rf_pdiab_N,rf_pdiab_U,rf_pdiab_Y,rf_gdiab_N,rf_gdiab_U,rf_gdiab_Y,rf_phype_N,rf_phype_U,rf_phype_Y,rf_ghype_N,rf_ghype_U,rf_ghype_Y,rf_ehype_N,rf_ehype_U,rf_ehype_Y,rf_ppterm_N,rf_ppterm_U,rf_ppterm_Y,rf_inftr_N,rf_inftr_U,rf_inftr_Y,rf_fedrg_N,rf_fedrg_U,rf_fedrg_X,rf_fedrg_Y,rf_artec_N,rf_artec_U,rf_artec_X,rf_artec_Y,rf_cesar_N,rf_cesar_U,rf_cesar_Y,rf_cesarn_Multi,no_risks_U,ip_gon_N,ip_gon_U,ip_gon_Y,ip_syph_N,ip_syph_U,ip_syph_Y,ip_chlam_N,ip_chlam_U,ip_chlam_Y,ip_hepatb_N,ip_hepatb_U,ip_hepatb_Y,ip_hepatc_N,ip_hepatc_U,ip_hepatc_Y,no_infec_U,ob_ecvs_N,ob_ecvs_U,ob_ecvs_Y,ob_ecvf_N,ob_ecvf_U,ob_ecvf_Y,ld_indl_N,ld_indl_U,ld_indl_Y,ld_augm_N,ld_augm_U,ld_augm_Y,ld_ster_N,ld_ster_U,ld_ster_Y,ld_antb_N,ld_antb_U,ld_antb_Y,ld_chor_N,ld_chor_U,ld_chor_Y,ld_anes_N,ld_anes_U,ld_anes_Y,no_lbrdlv_U,me_pres_Unknown,me_rout_Unknown,me_trial_N,me_trial_U,me_trial_X,me_trial_Y,rdmeth_rec_Unknown,mm_mtr_N,mm_mtr_U,mm_mtr_Y,mm_plac_N,mm_plac_U,mm_plac_Y,mm_rupt_N,mm_rupt_U,mm_rupt_Y,mm_uhyst_N,mm_uhyst_U,mm_uhyst_Y,mm_aicu_N,mm_aicu_U,mm_aicu_Y,no_mmorb_Unknown,attend_Unknown,mtran_N,mtran_U,mtran_Y,pay_Unknown,sex_F,sex_M
259150,1.244568,0.250795,0.076151,-0.337029,0.117525,2.685990,-0.707487,-0.045165,-0.184992,-0.141521,-0.709254,-1.418415,-0.183371,0.161612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
478985,0.381960,0.094156,0.744903,-0.337029,0.827999,-0.535588,0.422949,-0.045165,-0.143717,-0.069141,0.021195,0.220967,-0.183371,0.573095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
45564,1.417090,3.540231,-0.271599,-0.337029,-0.251922,1.397359,-0.707487,0.660315,-0.622506,-0.696432,-0.733602,-0.082622,-0.183371,-0.249871,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
399467,0.727003,0.250795,0.102901,-0.337029,0.145944,0.753043,1.553386,-0.750645,-0.531701,-0.913571,-0.977085,-0.143340,-0.183371,1.3

In [70]:
lassoCV_model = LassoCV(cv=5, random_state=42, verbose=1)
lassoCV_model.fit(_X_train_merged, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   44.5s finished


LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=42, selection='cyclic',
        tol=0.0001, verbose=1)

In [71]:
lassoCV_r2 = lassoCV_model.score(_X_test_merged, y_test)
lassoCV_pred = lassoCV_model.predict(_X_test_merged)
lassoCV_rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
print('R2 of LassoCV: ', lassoCV_r2)
print('RMSE of LassoCV: ', lassoCV_rmse)

R2 of LassoCV:  0.4165532614889236
RMSE of LassoCV:  444.6581800646426


In [72]:
coef_df = pd.DataFrame()
coef_df['features'] = _X_train_merged.columns
coef_df['values'] = lassoCV_model.coef_

In [73]:
coef_df.sort_values(by='values')

,features,values
118,ld_ster_Y,-2.764053e+02
155,mtran_Y,-1.250016e+02
62,rf_phype_Y,-1.208260e+02
157,sex_F,-1.189516e+02
68,rf_ehype_Y,-1.102786e+02
12,dplural,-1.020618e+02
54,rf_pdiab_N,-8.790003e+01
71,rf_ppterm_Y,-7.620241e+01
83,rf_cesar_N,-6.991734e+01
57,rf_gdiab_N,-5.429498e+01


In [74]:
lassoCV_model.intercept_

3293.6435260053095

In [42]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=0.1)
lasso_model.fit(_X_train_merged, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [44]:
lasso_r2 = lasso_model.score(_X_test_merged, y_test)
lasso_pred = lasso_model.predict(_X_test_merged)
lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
print('R2 of LassoCV: ', lasso_r2)
print('RMSE of LassoCV: ', lasso_rmse)

R2 of LassoCV:  0.41724195066446534
RMSE of LassoCV:  444.6581800646426


In [45]:
import optuna

In [108]:
selector = SelectKBest(f_regression, k=30)
selector.fit(_X_merged, y)

SelectKBest(k=30, score_func=<function f_regression at 0x000000001217B4C8>)

In [109]:
selected_columns = _X_merged.columns[selector.get_support()]

In [110]:
lm_kbest = LinearRegression()
lm_kbest = lm_kbest.fit(_X_merged[selected_columns], y)

In [111]:
print(lm_kbest.intercept_)
lm_kbest.score(_X_merged[selected_columns], y)

-1544.0562605481973


0.43279600193633416

In [112]:
coef_df = pd.DataFrame()
coef_df['variables'] = selected_columns
coef_df['values'] = lm_kbest.coef_
coef_df.sort_values(by='values')

,variables,values
5,dplural,-538.574534
22,ld_ster_Y,-391.765322
18,cig_3_Unknown,-292.732019
27,mtran_Y,-264.496841
9,mracehisp_NonHispanicBlack,-115.890299
20,rf_ghype_Y,-104.568796
23,me_pres_Breech,-93.680897
26,mtran_N,-60.108034
28,sex_F,-59.815841
10,mracehisp_NonHispanicWhite,-26.619169


In [153]:
_X_merged

,dob_tt,mager,fagecomb,illb_r,ilop_r,ilp_r,precare,previs,m_ht_in,bmi,pwgt_r,dwgt_r,wtgain,dplural,combgest,oegest_comb,dob_mm_1,dob_mm_2,dob_mm_3,dob_mm_4,dob_mm_5,dob_mm_6,dob_mm_7,dob_mm_8,dob_mm_9,dob_mm_10,dob_mm_11,dob_mm_12,dob_wk_1,dob_wk_2,dob_wk_3,dob_wk_4,dob_wk_5,dob_wk_6,dob_wk_7,bfacil_Unknown,mbstate_rec_Unknown,restatus_Foreign,mrace6_Multi,mhisp_r_Unknown,mracehisp_Unknown,dmar_Unmarried,meduc_Unknown,frace6_Unknown,fhisp_r_Unknown,fracehisp_Unknown,feduc_Unknown,priorlive_Multi,priordead_Multi,priorterm_Multi,cig_0_Yes,cig_1_Yes,cig_2_Yes,cig_3_Yes,f_m_ht_0,f_m_ht_1,rf_pdiab_N,rf_pdiab_U,rf_pdiab_Y,rf_gdiab_N,rf_gdiab_U,rf_gdiab_Y,rf_phype_N,rf_phype_U,rf_phype_Y,rf_ghype_N,rf_ghype_U,rf_ghype_Y,rf_ehype_N,rf_ehype_U,rf_ehype_Y,rf_ppterm_N,rf_ppterm_U,rf_ppterm_Y,rf_inftr_N,rf_inftr_U,rf_inftr_Y,rf_fedrg_N,rf_fedrg_U,rf_fedrg_X,rf_fedrg_Y,rf_artec_N,rf_artec_U,rf_artec_X,rf_artec_Y,rf_cesar_N,rf_cesar_U,rf_cesar_Y,rf_cesarn_Multi,no_risks_U,ip_gon_N,ip_gon_U,ip_gon_Y,ip_syph_N,ip_syph_U,ip_syph_Y,ip_chlam_N,ip_chlam_U,ip_chlam_Y,ip_hepatb_N,ip_hepatb_U,ip_hepatb_Y,ip_hepatc_N,ip_hepatc_U,ip_hepatc_Y,no_infec_U,ob_ecvs_N,ob_ecvs_U,ob_ecvs_Y,ob_ecvf_N,ob_ecvf_U,ob_ecvf_Y,ld_indl_N,ld_indl_U,ld_indl_Y,ld_augm_N,ld_augm_U,ld_augm_Y,ld_ster_N,ld_ster_U,ld_ster_Y,ld_antb_N,ld_antb_U,ld_antb_Y,ld_chor_N,ld_chor_U,ld_chor_Y,ld_anes_N,ld_anes_U,ld_anes_Y,no_lbrdlv_U,me_pres_Unknown,me_rout_Unknown,me_trial_N,me_trial_U,me_trial_X,me_trial_Y,rdmeth_rec_Unknown,mm_mtr_N,mm_mtr_U,mm_mtr_Y,mm_plac_N,mm_plac_U,mm_plac_Y,mm_rupt_N,mm_rupt_U,mm_rupt_Y,mm_uhyst_N,mm_uhyst_U,mm_uhyst_Y,mm_aicu_N,mm_aicu_U,mm_aicu_Y,no_mmorb_Unknown,attend_DoctorMedicine,attend_DoctorOsteopathy,attend_NurseMidwife,attend_Other,attend_OtherMidwife,attend_Unknown,mtran_N,mtran_U,mtran_Y,pay_IndianHealthService,pay_Medicaid,pay_Other,pay_OtherGovernment,pay_PrivateInsurance,pay_SelfPay,pay_Tricare,pay_Unknown,sex_F,sex_M
0,1227,30,31,16,33,16,3,8,66,30.7,190,231,41,1,38,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1704,35,35,180,0,180,3,9,63,33.3,188,185,0,1,39,39,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,938,23,26,43,0,43,5,6,64,23.7,138,138,0,1,31,39,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,28,26,28,39,0,39,2,13,67

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   45.9s finished


LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=42, selection='cyclic',
        tol=0.0001, verbose=1)

0.4575955703721287

In [36]:
coef_df = pd.DataFrame()
coef_df['features'] = _X_merged.columns
coef_df['values'] = lassoCV_model.coef_

In [37]:
coef_df.sort_values(by = 'values')

,features,values
132,cig_3_Unknown,-5.322999e+02
207,ld_ster_Y,-2.396552e+02
111,priorlive_None,-1.369377e+02
273,sex_F,-1.166576e+02
264,mtran_Y,-1.131762e+02
13,dplural,-1.088902e+02
136,rf_pdiab_N,-1.072654e+02
63,mracehisp_NonHispanicBlack,-1.049762e+02
150,rf_ehype_Y,-9.181363e+01
144,rf_phype_Y,-8.615725e+01


In [38]:
lassoCV_model.intercept_

3297.8450038923843